In [14]:
!pip install pandas seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
print(os.getcwd())

c:\Users\pcarv\Documents\projeto-incudaroa\analyse


In [ ]:
%pwd

'/content'

In [22]:
%cd sample_data/
%ls

[Errno 2] No such file or directory: 'sample_data/'
/content/sample_data
anscombe.json*                mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md*


In [ ]:
df = pd.read_csv('../data/data.csv', sep=",")
df.head(10)

,id,greenhous_temperature_celsius,greenhouse_humidity_percentage,greenhouse_illuminance_lux,online_temperature_celsius,online_humidity_percentage,greenhouse_total_volatile_organic_compounds_ppb,greenhouse_equivalent_co2_ppm,created,stress_risk
0,361496,13.223257,68.055576,0.0,7.3,89.0,370.000000,1080.444444,2021-03-03 20:10:04,0
1,361543,13.149615,68.271154,0.0,6.3,91.0,380.300000,1068.100000,2021-03-03 20:20:03,0
2,361595,13.094339,68.402229,0.0,6.3,91.0,371.000000,1022.700000,2021-03-03 20:30:04,0
3,361647,12.989395,68.483559,0.0,5.3,94.0,363.000000,995.600000,2021-03-03 20:40:03,0
4,361699,12.901927,68.556920,0.0,5.4,94.0,350.000000,960.888889,2021-03-03 20:50:05,0
5,361746,12.879110,68.746014,0.0,5.4,94.0,344.300000,943.300000,2021-03-03 21:00:05,0
6,361798,12.897269,69.032883,0.0,5.3,93.0,342.000000,938.900000,2021-03-03 21:10:03,0
7,361850,12.948807,69.289388,0.0,5.0,94.0,342.100000,935.000000,2021-03-03 21:20:06,0
8,361902,13.012004,69.456525,0.0,5.0,94.0,329.111111,921.666667,2021-03-03 21:30:07,0
9,361949,13.064699,69.512779,0.0,4.9,95.0,334.000000,928.200000,2021-03-03 21:40:03,0


## Adicionando uma feature para treino do modelo


In [10]:
cond1 = (df['greenhous_temperature_celsius'] > 32 ) & (df['greenhouse_humidity_percentage'] < 40)
cond2 = (df['greenhouse_equivalent_co2_ppm'] > 2000)
df['stress_risk'] = (cond1 | cond2).astype(int)

print("Valores ausentes:")
print(df.isnull().sum())
print("\n Estatísica Descritiva: ")
stats_summary = df.describe()
print(stats_summary)
stats_summary.to_csv('../extracted_features/stats_summary.csv')

Valores ausentes:
id                                                 0
greenhous_temperature_celsius                      0
greenhouse_humidity_percentage                     0
greenhouse_illuminance_lux                         0
online_temperature_celsius                         6
online_humidity_percentage                         6
greenhouse_total_volatile_organic_compounds_ppb    0
greenhouse_equivalent_co2_ppm                      0
created                                            0
stress_risk                                        0
dtype: int64

 Estatísica Descritiva: 
                 id  greenhous_temperature_celsius  \
count  1.716400e+04                   17164.000000   
mean   7.895066e+05                      25.874693   
std    2.441935e+05                      13.038214   
min    3.614960e+05                       4.972237   
25%    5.783872e+05                      14.951478   
50%    7.930140e+05                      24.121462   
75%    9.978682e+05                

In [ ]:
# Análise Univariada (Distribuições)
fig, axes = plt.subplots(2, 2, figsize=(14,10))
sns.histplot(df['greenhous_temperature_celsius'], ax=axes[0,0], color='salmon')
axes[0,0].set_title('Distribuição de Temperatura')

sns.histplot(df['greenhouse_humidity_percentage'], ax=axes[0,1], color='skyblue')
axes[0,1].set_title('Distribuição de Umidade')

sns.histplot(df['greenhouse_equivalent_co2_ppm'], ax=axes[1,0], color='lightgreen')
axes[1,0].set_title('Distribuição de CO2')

sns.histplot(df['online_temperature_celsius'], kde=True, ax=axes[1, 1], color='gray')
axes[1, 1].set_title('Distribuição da Temperatura (Online/Externa)')

plt.tight_layout()
plt.savefig("../extracted_features/univariante.png")
plt.close()

# Análise Bivariada
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df,
    x='greenhous_temperature_celsius',
    y='greenhouse_humidity_percentage',
    hue='stress_risk',
    style='stress_risk',
    palette={0: 'blue', 1: 'red'},
    markers={0: 'o', 1: 's'},
    s=100,
    alpha=0.7
)
plt.title('Risco de Estresse por Temperatura e Umidade')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Umidade (%)')
plt.legend(title='Risco de Estresse', loc='upper right')
plt.savefig("../extracted_features/bivariante.png")
plt.close()

# 4. Análise Multivariada (Correlação)
plt.figure(figsize=(10, 8))
num_df = df.select_dtypes(include=['float64', 'int64', 'int32']).drop(columns=['id', 'stress_risk'])
corr_matrix = num_df.corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1, linewidths=0.5)
plt.title('Multivariada: Matriz de Correlação')
plt.tight_layout()
plt.savefig('../extracted_features/3_multivariada.png')
plt.close()

print("\nImagens geradas com sucesso.")


Imagens geradas com sucesso.


In [17]:
sns.pairplot(df, hue='stress_risk', palette={0: 'blue', 1: 'red'}, markers=['o', 's'])
plt.title('Multivariada: Pair Plot')
plt.tight_layout()
plt.savefig('../extracted_features/4_pairplot.png')
plt.close()
